# PyTorch基础的神经网络

PyTorch实现神经网络，可以直接定义计算图，也可以通过Sequential和Module这两种方式实现

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

## 1 Sequential方式构建网络 

In [2]:
# Sequential 实现感知器模型
seq_net = nn.Sequential(
    nn.Linear(10, 1), # PyTorch 中的线性层，wx + b
    #nn.Tanh(),
    #nn.Sigmoid()
    nn.ReLU()
)

In [3]:
w0 = seq_net[0].weight
print(w0)

Parameter containing:
 0.0078 -0.3088 -0.2082 -0.1878 -0.1752  0.0058  0.2381 -0.1759  0.0472  0.2733
[torch.FloatTensor of size 1x10]



In [4]:
# Sequential 实现多层感知器
seq_net = nn.Sequential(
    nn.Linear(3, 5), 
    nn.Tanh(),
    nn.Linear(5, 5), 
    nn.Tanh(),
    nn.Linear(5, 5), 
    nn.Tanh(),
    nn.Linear(5, 1)
)

In [5]:
# 序列模块可以通过索引访问每一层

seq_net[0] # 第一层

Linear(in_features=3, out_features=5)

In [6]:
# 打印出第一层的权重

w0 = seq_net[0].weight
print(w0)

Parameter containing:
 0.3463  0.1165  0.1723
 0.1360  0.5471  0.1201
 0.4564 -0.2405  0.4262
 0.2911  0.4240  0.3968
 0.1698  0.0742  0.1217
[torch.FloatTensor of size 5x3]



In [7]:
w2 = seq_net[2].weight
print(w2)

Parameter containing:
-0.1871  0.3705  0.1985  0.1991  0.2747
 0.1674  0.0007 -0.4288 -0.4191  0.3472
-0.0014  0.3323 -0.1023  0.0680 -0.1485
-0.1783  0.0371 -0.3417 -0.3169 -0.0142
-0.0958  0.0971  0.0363 -0.4168 -0.3189
[torch.FloatTensor of size 5x5]



In [8]:
w4 = seq_net[4].weight
print(w4)

Parameter containing:
-0.2645 -0.4347  0.3349 -0.0560 -0.1670
 0.2548  0.0192 -0.2832  0.4415  0.1045
 0.0704 -0.3015  0.3710  0.0073  0.1018
 0.0112  0.2247  0.0223  0.4304 -0.3112
 0.1221 -0.3413  0.0872  0.1487 -0.2444
[torch.FloatTensor of size 5x5]



In [9]:
w6 = seq_net[6].weight
print(w6)

Parameter containing:
 0.0985 -0.1613  0.3832  0.1263 -0.2239
[torch.FloatTensor of size 1x5]



## 2 Module方式构建网络

In [10]:
# Module实现多层感知器
class module_net(nn.Module):
    def __init__(self, num_input, num_hidden, num_output):
        super(module_net, self).__init__()
        self.layer0 = nn.Linear(num_input, num_hidden)
        self.layer1 = nn.Tanh()
        self.layer2 = nn.Linear(num_hidden, num_hidden)
        self.layer3 = nn.Tanh()
        self.layer4 = nn.Linear(num_hidden, num_hidden)
        self.layer5 = nn.Tanh()
        self.layer6 = nn.Linear(num_hidden, num_output)
        
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x

In [11]:
mo_net = module_net(3, 5, 1)

In [12]:
# 访问模型中的某层可以直接通过名字

# 第一层
w0 = mo_net.layer0
print(w0)

Linear(in_features=3, out_features=5)


In [13]:
# 打印出第一层的权重
print(w0.weight)

Parameter containing:
 0.4627 -0.3294 -0.4896
 0.0824 -0.0297 -0.2998
 0.1473 -0.0720 -0.4867
-0.5267 -0.3887  0.2535
 0.4469 -0.3929 -0.3655
[torch.FloatTensor of size 5x3]



## 3 模型保存与读取

In [14]:
# 将参数和模型保存在一起
torch.save(mo_net, './save_mo_net.pth')

/Library/Python/2.7/site-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type module_net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [15]:
# 读取保存的模型
mo_net1 = torch.load('./save_mo_net.pth')